In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Custom training and online prediction

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/sdk-custom-image-classification-online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fcustom%2Fsdk-custom-image-classification-online.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/custom/sdk-custom-image-classification-online.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/sdk-custom-image-classification-online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

## Overview


This tutorial demonstrates how to use the Vertex AI SDK for Python to train and deploy a custom image classification model for online prediction.

Learn more about [Custom training](https://cloud.google.com/vertex-ai/docs/training/custom-training) and [Vertex AI Prediction](https://cloud.google.com/vertex-ai/docs/predictions/get-predictions).

### Objective

In this tutorial, you learn to use `Vertex AI Training` to create a custom-trained model from a Python script in a Docker container, and learn to use `Vertex AI Prediction` to do a prediction on the deployed model by sending data. Alternatively, you can create custom-trained models using `gcloud` command-line tool, or online using the Cloud Console.

This tutorial uses the following Google Cloud ML services:

- `Vertex AI Training`
- `Vertex AI Prediction`
- `Vertex AI Model` resource
- `Vertex AI Endpoint` resource

The steps performed include:

- Create a `Vertex AI` custom job for training a TensorFlow model.
- Upload the trained model artifacts to a `Model` resource.
- Create a serving `Endpoint` resource.
- Deploy the `Model` resource to a serving `Endpoint` resource.
- Make a prediction.
- Undeploy the `Model` resource.

### Dataset

The dataset used for this tutorial is the [cifar10 dataset](https://www.tensorflow.org/datasets/catalog/cifar10) from [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview). The version of the dataset you will use is built into TensorFlow. The trained model predicts which type of class an image is from ten classes: airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck.

### Costs

This tutorial uses billable components of Google Cloud (GCP):

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Get started

### Install Vertex AI SDK for Python and other required packages


In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud \
                                 google-cloud-storage \
                                 pillow \
                                 numpy

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK for Python

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

#### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**If your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $LOCATION -p $PROJECT_ID $BUCKET_URI

#### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [8]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### Import libraries and define constants

In [9]:
import os

import numpy as np
from PIL import Image

In [10]:

from google import genai
from google.genai.types import HttpOptions

client = genai.Client(http_options=HttpOptions(api_version="v1"))
response = client.models.compute_tokens(
    model="gemini-2.0-flash-001",
    contents="What's the longest word in the English language?",
)
print(response)

ValueError: Missing key inputs argument! To use the Google AI API,provide (`api_key`) arguments. To use the Google Cloud API, provide (`vertexai`, `project` & `location`) arguments.

In [12]:

from vertexai.preview.tokenization import get_tokenizer_for_model

# init local tokenzier
tokenizer = get_tokenizer_for_model("gemini-1.5-flash-001")

# Count Tokens
prompt = """'When do you want this change to take effect: select the effective date of the transfer.\n\nWhy are you making this change: considering you are requesting a transfer, select "Transfer Contingent Worker" \n\nOnce all is completed, click "Start". A new page will open, review all the data and, if all OK, click "Submit". \n\nThe process will now flow to the current manager for them to approve the transfer:\n\nStep 3 - Current Manager review and approval\n\nAs the current manager, you will receive a notification about the request for transfer in their Workday Inbox. It can either be found on the HomePage:\n\nOr, via the My Tasks button on the top-right corner:\n\nStep 3.1 - Start Details\n\nAfter clicking on the task, a page with the Review action appears. The new data is marked with a blue circle on the left, and the old/removed data is marked with a red X.\n\nStep 3.2 - Position Management\n\n????????/??????????\n\n\n\nThe rest items in Start keep as is\n\n???????\n\n\n\n\n\n\n\n\n\nClick Start\n\n??Start\n\n\n\nDetails ????\n\n\n\nFind ?End Employment Date? and input new labor contract /agreement end date\n\n??????????????????/???????\n\n\n\nStart date: 2nd January- 1st July (include), End date should be 30th June 3 years later;\n\nStart date: 2nd July- 1st January in next year (include), End date should be 31st December 3 years later.\n\n?????/???????1?2?-7?1?????????????????6?30??\n\n?????/???????7?2?-??1?1?????????????????12?31??\n\n\n\ne.g., Start date: 2024/01/02 - 2024/07/01, \n\n         End date=2027/06/30 \n\n\n\n         Start date: 2024/07/02-2025/01/01, \n\n         End date=2027/12/31\n\n\n\n\n\nNote ???\n\nDo not edit any other information.\n\n????????\n\n\n\n\n\nAttachments ??\n\n\n\nNo need attachment\n\n??????\n\n\n\n\n\nYou can enter comments at the bottom.\n\n???????????????\n\n\n\nClick Submit and the process will be done after approving by relevant parties.\n\n?????????????????\n\n??Workday??????? ??->????->?????? ????\n\n\n\n\n\nSelect change reason: Lateral Move\n\n?????????\n\n\n\n\n\nInput effective date ??????\n\n\n\nNote: Effective date should be the day next to employee?s legal retirement day.\n\n????????????????(??)?????\n\n\n\n\n\nUse Next Pay Period: keep as is\n\n?????????????: ????\n\n\n\nClick on Start ?? ??\n\n\n\n\n\nPosition: Keep as-is.\n\n?? ????\n\n\n\n(Optional) Maintain Job Profile and Job Title ?????? ?????\n\n\n\nJob Profile:\n\n\n\nClick Job Profile to select new Job Profile if needed, otherwise keep it as is.\n\n???????????, ?????????????????\n\n\n\nJob Title :\n\n(Job Title will be populated automatically based on Job Profile, it will be reflected in Teams/Outlook. Please ensure Job Level is removed from Job Title\n\n???????????????????Outlook?Teams????????????????)\n\n\n\nClick Job Title to input new Job Title, remove Job Level from Job Title\n\n??????, ?????????????\n\n\n\n\n\n(Optional) Maintain Location ????\n\n\n\nUpdate Location information if need, otherwise keep as is.\n\n????????,???????\n\n\n\nNote:\n\nStep 2 - Initiate Job Change\n\nGo to the Contingent Worker\'s profile and follow the path: Actions > Job Change > Start Job Change.\n\nA new screen will open to initiate the Job Change actions. Here you can see the name of the contingent worker, the job profile, and the action you need to take (please note that "Change Contingent Worker Details" is the only option available).\n\nClick "OK".\n\nStep 2.1 - Start Details\n\nSelect the effective date in the field, "When do you want this change to take effect?". The effective date is typically the following day of the current Contract End Date.\n\nStep 2.2 - Contract Details\n\nSelect the updated "Contract End Date" (please note that the new contract period should not be longer than 1 year).\n\nAdditional information in FAQ format\n\nHow to get reports for all the externals in my function?\n\nYou can raise a case via Workday\n\nI started the worker\'s extension after the contract end date and their account has already been disabled, what do I need to do?\n\nUse Next Pay Period: keep as is\n\n?????????????:????\n\n\n\nClick on Start ????\n\n\n\n\n\nPosition: Keep as-is.\n\n?? ????\n\n\n\n(Optional) Maintain Job Profile and Job Title???????????\n\n\n\nJob Profile:\n\n\n\nClick Job Profile to select new Job Profile if needed, otherwise keep it as is.\n\n???????????, ?????????????????\n\n\n\nJob Title :\n\n(Job Title will be populated automatically based on Job Profile, it will be reflected in Teams/Outlook. Please ensure Job Level is removed from Job Title\n\n???????????????????Outlook?Teams????????????????)\n\n\n\nClick Job Title to input new Job Title, remove Job Level from Job Title\n\n??????, ?????????????\n\n\n\n\n\n(Optional) Maintain Location ????\n\n\n\nUpdate Location information if need, otherwise keep as is.\n\n????????,???????\n\n\n\nEdit Schedule weekly hours?40\n\n?????? ????40\n\n\n\n(Optional)\n\nMaintain Organization information????????\n\n\n\nEdit Company, Cost Center, Brand if need, otherwise keep as is. ????????,????,??, ???????'"""
response = tokenizer.count_tokens(prompt)
print(f"Tokens count: {response.total_tokens}")
# Example response:
#       Tokens count: 6

# Compute Tokens
response = tokenizer.compute_tokens(prompt)
print(f"Tokens list: {response.tokens_info}")

Tokens count: 1142
Tokens list: [TokensInfo(token_ids=[235303, 4115, 749, 692, 1938, 736, 2669, 577, 1987, 4232, 235292, 6616, 573, 7017, 3209, 576, 573, 5853, 235265, 109, 4385, 708, 692, 3547, 736, 2669, 235292, 14113, 692, 708, 51174, 476, 5853, 235269, 6616, 664, 22547, 129582, 579, 52167, 235281, 235248, 109, 14326, 832, 603, 8735, 235269, 5499, 664, 5089, 2776, 586, 888, 2602, 877, 2174, 235269, 3221, 832, 573, 1423, 578, 235269, 1013, 832, 11530, 235269, 5499, 664, 14921, 2776, 235248, 109, 651, 2185, 877, 1490, 3781, 577, 573, 2474, 8866, 604, 1174, 577, 35344, 573, 5853, 235292, 109, 5856, 235248, 235304, 728, 12344, 10713, 3221, 578, 15560, 109, 2169, 573, 2474, 8866, 235269, 692, 877, 6427, 476, 21244, 1105, 573, 3853, 604, 5853, 575, 1024, 5624, 1311, 170766, 235265, 1165, 798, 3906, 614, 1942, 611, 573, 94966, 235292, 109, 2556, 235269, 3998, 573, 2961, 63960, 6086, 611, 573, 2267, 235290, 1331, 10429, 235292, 109, 5856, 235248, 235304, 235265, 235274, 728, 7248, 9579, 109

In [14]:

response.tokens_info[0]

TokensInfo(token_ids=[235303, 4115, 749, 692, 1938, 736, 2669, 577, 1987, 4232, 235292, 6616, 573, 7017, 3209, 576, 573, 5853, 235265, 109, 4385, 708, 692, 3547, 736, 2669, 235292, 14113, 692, 708, 51174, 476, 5853, 235269, 6616, 664, 22547, 129582, 579, 52167, 235281, 235248, 109, 14326, 832, 603, 8735, 235269, 5499, 664, 5089, 2776, 586, 888, 2602, 877, 2174, 235269, 3221, 832, 573, 1423, 578, 235269, 1013, 832, 11530, 235269, 5499, 664, 14921, 2776, 235248, 109, 651, 2185, 877, 1490, 3781, 577, 573, 2474, 8866, 604, 1174, 577, 35344, 573, 5853, 235292, 109, 5856, 235248, 235304, 728, 12344, 10713, 3221, 578, 15560, 109, 2169, 573, 2474, 8866, 235269, 692, 877, 6427, 476, 21244, 1105, 573, 3853, 604, 5853, 575, 1024, 5624, 1311, 170766, 235265, 1165, 798, 3906, 614, 1942, 611, 573, 94966, 235292, 109, 2556, 235269, 3998, 573, 2961, 63960, 6086, 611, 573, 2267, 235290, 1331, 10429, 235292, 109, 5856, 235248, 235304, 235265, 235274, 728, 7248, 9579, 109, 5514, 24212, 611, 573, 6911, 23

### Set pre-built containers

Vertex AI provides pre-built containers to run training and prediction.

For the latest list, see [Pre-built containers for training](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers) and [Pre-built containers for prediction](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)

In [ ]:
TRAIN_VERSION = "tf-cpu.2-9"
DEPLOY_VERSION = "tf2-cpu.2-9"

TRAIN_IMAGE = "us-docker.pkg.dev/vertex-ai/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "us-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(DEPLOY_VERSION)

# Tutorial

Now you are ready to start creating your own custom-trained model with CIFAR10.
## Train a model

There are two ways you can train a custom model using a container image:

- **Use a Google Cloud prebuilt container**. If you use a prebuilt container, you will additionally specify a Python package to install into the container image. This Python package contains your code for training a custom model.

- **Use your own custom container image**. If you use your own container, the container needs to contain your code for training a custom model.

### Define the command args for the training script

Prepare the command-line arguments to pass to your training script.
- `args`: The command line arguments to pass to the corresponding Python module. In this example, they will be:
  - `"--epochs=" + EPOCHS`: The number of epochs for training.
  - `"--steps=" + STEPS`: The number of steps (batches) per epoch.
  - `"--distribute=" + TRAIN_STRATEGY"` : The training distribution strategy to use for single or distributed training.
     - `"single"`: single device.
     - `"mirror"`: all GPU devices on a single compute instance.
     - `"multi"`: all GPU devices on all compute instances.

In [ ]:
JOB_NAME = "custom_job_unique"
MODEL_DIR = "{}/{}".format(BUCKET_URI, JOB_NAME)

TRAIN_STRATEGY = "single"

EPOCHS = 20
STEPS = 100

CMDARGS = [
    "--epochs=" + str(EPOCHS),
    "--steps=" + str(STEPS),
    "--distribute=" + TRAIN_STRATEGY,
]

#### Training script

In the next cell, you will write the contents of the training script, `task.py`. In summary:

- Get the directory where to save the model artifacts from the environment variable `AIP_MODEL_DIR`. This variable is set by the training service.
- Loads CIFAR10 dataset from TF Datasets (tfds).
- Builds a model using TF.Keras model API.
- Compiles the model (`compile()`).
- Sets a training distribution strategy according to the argument `args.distribute`.
- Trains the model (`fit()`) with epochs and steps according to the arguments `args.epochs` and `args.steps`
- Saves the trained model (`save(MODEL_DIR)`) to the specified model directory.

In [ ]:
%%writefile task.py
# Single, Mirror and Multi-Machine Distributed Training for CIFAR-10

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--lr', dest='lr',
                    default=0.01, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print('DEVICES', device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64

def make_datasets_unbatched():
  # Scaling CIFAR10 data from (0, 255] to (0., 1.]
  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label

  datasets, info = tfds.load(name='cifar10',
                            with_info=True,
                            as_supervised=True)
  return datasets['train'].map(scale).cache().shuffle(BUFFER_SIZE).repeat()


# Build the Keras model
def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      optimizer=tf.keras.optimizers.SGD(learning_rate=args.lr),
      metrics=['accuracy'])
  return model

# Train the model
NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS
MODEL_DIR = os.getenv("AIP_MODEL_DIR")

train_dataset = make_datasets_unbatched().batch(GLOBAL_BATCH_SIZE)

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_cnn_model()

model.fit(x=train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(MODEL_DIR)

### Train the model

Define your custom training job on Vertex AI.

Use the `CustomTrainingJob` class to define the job, which takes the following parameters:

- `display_name`: The user-defined name of this training pipeline.
- `script_path`: The local path to the training script.
- `container_uri`: The URI of the training container image.
- `requirements`: The list of Python package dependencies of the script.
- `model_serving_container_image_uri`: The URI of a container that can serve predictions for your model — either a prebuilt container or a custom container.

Use the `run` function to start training, which takes the following parameters:

- `args`: The command line arguments to be passed to the Python script.
- `replica_count`: The number of worker replicas.
- `model_display_name`: The display name of the `Model` if the script produces a managed `Model`.
- `machine_type`: The type of machine to use for training.
- `accelerator_type`: The hardware accelerator type.
- `accelerator_count`: The number of accelerators to attach to a worker replica.

The `run` function creates a training pipeline that trains and creates a `Model` object. After the training pipeline completes, the `run` function returns the `Model` object.

In [ ]:
job = aiplatform.CustomTrainingJob(
    display_name=JOB_NAME,
    script_path="task.py",
    container_uri=TRAIN_IMAGE,
    requirements=["tensorflow_datasets"],
    model_serving_container_image_uri=DEPLOY_IMAGE,
)

MODEL_DISPLAY_NAME = "cifar10_unique"

# Start the training
model = job.run(
    model_display_name=MODEL_DISPLAY_NAME,
    args=CMDARGS,
    replica_count=1,
)

### Deploy the model

Before you use your model to make predictions, you need to deploy it to an `Endpoint`. You can do this by calling the `deploy` function on the `Model` resource. This will do two things:

1. Create an `Endpoint` resource for deploying the `Model` resource to.
2. Deploy the `Model` resource to the `Endpoint` resource.


The function takes the following parameters:

- `deployed_model_display_name`: A human readable name for the deployed model.
- `traffic_split`: Percent of traffic at the endpoint that goes to this model, which is specified as a dictionary of one or more key/value pairs.
   - If only one model, then specify as **{ "0": 100 }**, where "0" refers to this model being uploaded and 100 means 100% of the traffic.
   - If there are existing models on the endpoint, for which the traffic will be split, then use `model_id` to specify as **{ "0": percent, model_id: percent, ... }**, where `model_id` is the model id of an existing model to the deployed endpoint. The percents must add up to 100.
- `machine_type`: The type of machine to use for training.
- `accelerator_type`: The hardware accelerator type.
- `accelerator_count`: The number of accelerators to attach to a worker replica.
- `starting_replica_count`: The number of compute instances to initially provision.
- `max_replica_count`: The maximum number of compute instances to scale to. In this tutorial, only one instance is provisioned.

### Traffic split

The `traffic_split` parameter is specified as a Python dictionary. You can deploy more than one instance of your model to an endpoint, and then set the percentage of traffic that goes to each instance.

You can use a traffic split to introduce a new model gradually into production. For example, if you had one existing model in production with 100% of the traffic, you could deploy a new model to the same endpoint, direct 10% of traffic to it, and reduce the original model's traffic to 90%. This allows you to monitor the new model's performance while minimizing the distruption to the majority of users.

### Compute instance scaling

You can specify a single instance (or node) to serve your online prediction requests. This tutorial uses a single node, so the variables `MIN_NODES` and `MAX_NODES` are both set to `1`.

If you want to use multiple nodes to serve your online prediction requests, set `MAX_NODES` to the maximum number of nodes you want to use. Vertex AI autoscales the number of nodes used to serve your predictions, up to the maximum number you set. Refer to the [pricing page](https://cloud.google.com/vertex-ai/pricing#prediction-prices) to understand the costs of autoscaling with multiple nodes.

### Endpoint

The method will block until the model is deployed and eventually return an `Endpoint` object. If this is the first time a model is deployed to the endpoint, it may take a few additional minutes to complete provisioning of resources.

In [ ]:
DEPLOYED_NAME = "cifar10_deployed_unique"

TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1


endpoint = model.deploy(
    deployed_model_display_name=DEPLOYED_NAME,
    traffic_split=TRAFFIC_SPLIT,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
)

## Make an online prediction request

Send an online prediction request to your deployed model.
### Get test data

Download images from the CIFAR dataset and preprocess them.

#### Download the test images

Download the provided set of images from the CIFAR dataset:

In [ ]:
# Download the images
! gsutil -m cp -r gs://cloud-samples-data/ai-platform-unified/cifar_test_images .

#### Preprocess the images
Before you can run the data through the endpoint, you need to preprocess it to match the format that your custom model defined in `task.py` expects.

`x_test`:
Normalize (rescale) the pixel data by dividing each pixel by 255. This replaces each single byte integer pixel with a 32-bit floating point number between 0 and 1.

`y_test`:
You can extract the labels from the image filenames. Each image's filename format is "image_{LABEL}_{IMAGE_NUMBER}.jpg"

In [ ]:
# Load image data
IMAGE_DIRECTORY = "cifar_test_images"

image_files = [file for file in os.listdir(IMAGE_DIRECTORY) if file.endswith(".jpg")]

# Decode JPEG images into numpy arrays
image_data = [
    np.asarray(Image.open(os.path.join(IMAGE_DIRECTORY, file))) for file in image_files
]

# Scale and convert to expected format
x_test = [(image / 255.0).astype(np.float32).tolist() for image in image_data]

# Extract labels from image name
y_test = [int(file.split("_")[1]) for file in image_files]

### Send the prediction request

Now that you have test images, you can use them to send a prediction request. Use the `Endpoint` object's `predict` function, which takes the following parameters:

- `instances`: A list of image instances. According to your custom model, each image instance should be a 3-dimensional matrix of floats. This was prepared in the previous step.

The `predict` function returns a list, where each element in the list corresponds to the corresponding image in the request. You will see in the output for each prediction:

- Confidence level for the prediction (`predictions`), between 0 and 1, for each of the ten classes.

You can then run a quick evaluation on the prediction results:
1. `np.argmax`: Convert each list of confidence levels to a label
2. Compare the predicted labels to the actual labels
3. Calculate `accuracy` as `correct/total`

In [ ]:
predictions = endpoint.predict(instances=x_test)
y_predicted = np.argmax(predictions.predictions, axis=1)

correct = sum(y_predicted == np.array(y_test))
accuracy = len(y_predicted)
print(
    f"Correct predictions = {correct}, Total predictions = {accuracy}, Accuracy = {correct/accuracy}"
)

## Undeploy the model

To undeploy your `Model` resource from the serving `Endpoint` resource, use the endpoint's `undeploy` method with the following parameter:

- `deployed_model_id`: The model deployment identifier returned by the endpoint service when the `Model` resource was deployed. You can retrieve the deployed models using the endpoint's `deployed_models` property.

Since this is the only deployed model on the `Endpoint` resource, you can omit `traffic_split`.

In [ ]:
deployed_model_id = endpoint.list_models()[0].id
endpoint.undeploy(deployed_model_id=deployed_model_id)

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Training Job
- Model
- Endpoint
- Cloud Storage Bucket

In [ ]:
# Warning: Setting this to true will delete everything in your bucket
delete_bucket = False

# Delete the training job
job.delete()

# Delete the model
model.delete()

# Delete the endpoint
endpoint.delete()

if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI